In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
base_dir = 'fruit_dataset'
train_dir = os.path.join(base_dir,  'Train')
test_dir = os.path.join(base_dir, 'Test')
for dirpath, dirnames, filenames in os.walk(train_dir):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 58 directories and 0 images in 'fruit_dataset\Train'.
There are 0 directories and 1169 images in 'fruit_dataset\Train\ackee'.
There are 0 directories and 1203 images in 'fruit_dataset\Train\apple'.
There are 0 directories and 1126 images in 'fruit_dataset\Train\apricot'.
There are 0 directories and 1169 images in 'fruit_dataset\Train\avocado'.
There are 0 directories and 1162 images in 'fruit_dataset\Train\banana'.
There are 0 directories and 1000 images in 'fruit_dataset\Train\bell pepper'.
There are 0 directories and 1022 images in 'fruit_dataset\Train\betel nut'.
There are 0 directories and 1077 images in 'fruit_dataset\Train\bitter gourd'.
There are 0 directories and 927 images in 'fruit_dataset\Train\black cherry'.
There are 0 directories and 1025 images in 'fruit_dataset\Train\black mullberry'.
There are 0 directories and 1002 images in 'fruit_dataset\Train\blueberry'.
There are 0 directories and 999 images in 'fruit_dataset\Train\bottle gourd'.
There are 0 directories 

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(test_dir,
                                                       target_size=(224, 224),
                                                       batch_size=32,
                                                       class_mode='categorical')


Found 56132 images belonging to 58 classes.
Found 15668 images belonging to 58 classes.


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# Load MobileNetV2 as base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model to retain the pre-trained weights
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)  # Dense layer for more complex features
predictions = Dense(58, activation='softmax')(x)  # Output layer for 58 classes

# Final model
model = Model(inputs=base_model.input, outputs=predictions)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 7s 1us/step


In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Save model after each epoch
checkpoint = ModelCheckpoint(
    filepath='model/fruit_model_epoch_{epoch:02d}.h5',  # Path to save
    save_weights_only=False,  # Save the entire model (architecture + weights)
    save_best_only=False,  # Save every epoch, not just the best one
    monitor='loss',  # Monitor training loss (or validation loss/accuracy)
    verbose=1
)


ValueError: The filepath provided must end in `.keras` (Keras model format). Received: filepath=model/fruit_model_epoch_{epoch:02d}.h5

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator, verbose=1)


Epoch 1/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6281 - loss: 1.3612

C:\Users\Irfan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1755/1755 ━━━━━━━━━━━━━━━━━━━━ 4198s 2s/step - accuracy: 0.6282 - loss: 1.3610 - val_accuracy: 0.7792 - val_loss: 0.7511
Epoch 2/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3889s 2s/step - accuracy: 0.7615 - loss: 0.8031 - val_accuracy: 0.8111 - val_loss: 0.6352
Epoch 3/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3862s 2s/step - accuracy: 0.7911 - loss: 0.6973 - val_accuracy: 0.8184 - val_loss: 0.6100
Epoch 4/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3853s 2s/step - accuracy: 0.8151 - loss: 0.6156 - val_accuracy: 0.8101 - val_loss: 0.6387
Epoch 5/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3918s 2s/step - accuracy: 0.8273 - loss: 0.5790 - val_accuracy: 0.8568 - val_loss: 0.4721
Epoch 6/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3908s 2s/step - accuracy: 0.8387 - loss: 0.5446 - val_accuracy: 0.8557 - val_loss: 0.4907
Epoch 7/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3901s 2s/step - accuracy: 0.8478 - loss: 0.5052 - val_accuracy: 0.8625 - val_loss: 0.4568
Epoch 8/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3686s 2s/step - accuracy: 0.8533 - loss: 0.48

In [11]:
# Unfreeze the last 10 layers of MobileNetV2 for fine-tuning
for layer in base_model.layers[-10:]:
    layer.trainable = True

# Re-compile and continue training
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

history_finetune = model.fit(train_generator, epochs=5, validation_data=validation_generator, verbose=1)


Epoch 1/5
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 4131s 2s/step - accuracy: 0.7974 - loss: 0.6930 - val_accuracy: 0.8391 - val_loss: 0.5822
Epoch 2/5
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3104s 2s/step - accuracy: 0.8580 - loss: 0.4673 - val_accuracy: 0.8618 - val_loss: 0.5204
Epoch 3/5
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3525s 2s/step - accuracy: 0.8760 - loss: 0.4063 - val_accuracy: 0.8821 - val_loss: 0.4196
Epoch 4/5
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3357s 2s/step - accuracy: 0.8886 - loss: 0.3583 - val_accuracy: 0.9039 - val_loss: 0.3361
Epoch 5/5
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 2604s 1s/step - accuracy: 0.8946 - loss: 0.3408 - val_accuracy: 0.9075 - val_loss: 0.3286


In [15]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os

# Ensure directories exist
os.makedirs('models', exist_ok=True)

# Early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Checkpoint to save the best model
checkpoint = ModelCheckpoint(filepath='models/best_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model with callbacks
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[early_stopping, checkpoint],
    verbose=1
)

# Evaluate the model
score = model.evaluate(validation_generator)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

# Save the final model
model.save('models/fruit_classifier_model.h5')



Epoch 1/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9000 - loss: 0.3165
Epoch 1: val_accuracy improved from -inf to 0.89763, saving model to models/best_model.keras
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3901s 2s/step - accuracy: 0.9000 - loss: 0.3165 - val_accuracy: 0.8976 - val_loss: 0.3652
Epoch 2/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9079 - loss: 0.2973
Epoch 2: val_accuracy improved from 0.89763 to 0.90388, saving model to models/best_model.keras
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3354s 2s/step - accuracy: 0.9079 - loss: 0.2973 - val_accuracy: 0.9039 - val_loss: 0.3459
Epoch 3/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9124 - loss: 0.2834
Epoch 3: val_accuracy improved from 0.90388 to 0.91199, saving model to models/best_model.keras
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 3807s 2s/step - accuracy: 0.9124 - loss: 0.2834 - val_accuracy: 0.9120 - val_loss: 0.3290
Epoch 4/10
1755/1755 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.9158 - loss: 0.2641
Epoc

Validation loss: 0.25174808502197266
Validation accuracy: 0.9308143854141235


In [20]:
from tensorflow.keras.models import load_model

# Load your saved model
model = load_model('models/best_model.keras')

from tensorflow.keras.utils import load_img, img_to_array
import numpy as np

# Load and preprocess the image
image_path = 'papaya.jpg'  # Replace with the path to your test image
img = load_img(image_path, target_size=(224, 224))  # Resize to the target input size
img_array = img_to_array(img) / 255.0  # Convert image to array and normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make prediction
predictions = model.predict(img_array)

# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])

# Get class labels (assuming you have used a data generator like ImageDataGenerator)
class_labels = {v: k for k, v in train_generator.class_indices.items()}  # Reverse the mapping

# Get the predicted label
predicted_class = class_labels[predicted_class_index]
print(f"Predicted class: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted class: papaya


In [ ]:
score = model.evaluate(validation_generator)
print('Validation loss:', score[0])
print('Validation accuracy:', score[1])

model.save('fruit_classifier_model.h5')